### Forward and Simulation
* Read forward solution
* Read stored simulation

In [1]:
from forward import solve_forward
from simulation import Simulation
import numpy as np

In [2]:
fwd = solve_forward()

# load eeg and source data from previous simulations
eeg_data = np.load('/media/thanos/Big Data/Thanos/TUC/Thesis/sim_data/sim_type_1/eeg_big_sim_2.npy')
source_data = np.load('/media/thanos/Big Data/Thanos/TUC/Thesis/sim_data/sim_type_1/sources_big_sim_2.npy')
sim = Simulation(fwd=fwd,source_data=source_data,eeg_data=eeg_data)

Forward problem is solved.
Electrodes: (73, 3)
Dipoles: (50460, 3)
Leadfield: (73, 50460)


### Train the Neural Network
Create and train an MLP network

In [3]:
from net import EEGMLP

eegnet = EEGMLP(sim=sim)

2021-12-04 11:41:56.181753: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-04 11:41:56.181788: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
eegnet.load_nn('/media/thanos/Big Data/Thanos/TUC/Thesis/nn_trained/mlp_model_2.h5')

2021-12-04 11:42:48.687175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-04 11:42:48.688338: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-04 11:42:48.689369: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thanos): /proc/driver/nvidia/version does not exist
2021-12-04 11:42:48.704222: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-04 11:42:49.261498: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 206684160 exceeds 10

Loaded model in EEGMLP :
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-1 (Dense)            (None, 256)               18944     
                                                                 
 Hidden-2 (Dense)            (None, 512)               131584    
                                                                 
 Hidden-3 (Dense)            (None, 1024)              525312    
                                                                 
 Output (Dense)              (None, 50460)             51721500  
                                                                 
Total params: 52,397,340
Trainable params: 52,397,340
Non-trainable params: 0
_________________________________________________________________


2021-12-04 11:42:54.057606: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 206684160 exceeds 10% of free system memory.


In [ ]:
eegnet.build_model()

In [ ]:
eegnet.fit()

In [ ]:
eegnet.save_nn_model('/media/thanos/Big Data/Thanos/TUC/Thesis/nn_trained/mlp_model_2.h5')

### Predict 
Predict the sources from other simulations using the trained neural network

In [5]:
eeg_data = np.load('/media/thanos/Big Data/Thanos/TUC/Thesis/sim_data/sim_type_2/eeg_big_sim_1.npy')
source_data = np.load('/media/thanos/Big Data/Thanos/TUC/Thesis/sim_data/sim_type_2/sources_big_sim_1.npy')

In [6]:
eeg = eeg_data[:,1:3]

In [7]:
predicted_sources = eegnet.predict(eeg.T).T

In [8]:
sources = source_data[:,1:3]

In [9]:
from sklearn.metrics import mean_squared_error

mean_squared_error(sources[:,0],predicted_sources[:,0])

59.03265381502554

In [19]:
eeg = sim.eeg_data[:,1:3]
predicted_sources = eegnet.predict(eeg.T).T
sources = sim.source_data[:,1:3]
mean_squared_error(sources[:,0],predicted_sources[:,0])

58.7576503761292